In [2]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from keras.layers.advanced_activations import LeakyReLU
from keras.models import Sequential, Model, load_model
from keras.layers import Activation, Convolution2D, MaxPooling2D, BatchNormalization, Flatten, Dense, Dropout, Conv2D,MaxPool2D, ZeroPadding2D
from keras.callbacks import ModelCheckpoint, EarlyStopping

In [3]:


#os.environ['KAGGLE_USERNAME'] = "mratel" # username from the json file
#os.environ['KAGGLE_KEY'] = "2f57b686988cdfa59b5092b2ea2fc8be" # key from the json file

!kaggle competitions download -c facial-keypoints-detection

train = pd.read_csv('training.zip')
test = pd.read_csv('test.zip')
lookid_data = pd.read_csv('IdLookupTable.csv')

#train.T.isnull().any()

train.fillna(method = 'ffill',inplace = True)

imag = []
for i in range(0,7049):
    img = train['Image'][i].split(' ')
    img = ['0' if x == '' else x for x in img]
    imag.append(img)

image_list = np.array(imag,dtype = 'float')

X_train = image_list.reshape(-1,96,96,1)

plt.imshow(X_train[0].reshape(96,96),cmap='gray')
plt.show()

training = train.drop('Image',axis = 1)

y_train = []
for i in range(0,7049):
    y = training.iloc[i,:]

    y_train.append(y)
y_train = np.array(y_train,dtype = 'float')

Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 5, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python2.7/dist-packages/kaggle/__init__.py", line 23, in <module>
    api.authenticate()
  File "/usr/local/lib/python2.7/dist-packages/kaggle/api/kaggle_api_extended.py", line 146, in authenticate
    self.config_file, self.config_dir))
IOError: Could not find kaggle.json. Make sure it's located in /root/.kaggle. Or use the environment method.


FileNotFoundError: ignored

In [ ]:
model = Sequential()

model.add(Convolution2D(32, (3,3), padding='same', use_bias=False, input_shape=(96,96,1)))
model.add(LeakyReLU(alpha = 0.1))
model.add(BatchNormalization())

model.add(Convolution2D(32, (3,3), padding='same', use_bias=False))
model.add(LeakyReLU(alpha = 0.1))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2, 2)))

model.add(Convolution2D(64, (3,3), padding='same', use_bias=False))
model.add(LeakyReLU(alpha = 0.1))
model.add(BatchNormalization())

model.add(Convolution2D(64, (3,3), padding='same', use_bias=False))
model.add(LeakyReLU(alpha = 0.1))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2, 2)))

model.add(Convolution2D(96, (3,3), padding='same', use_bias=False))
model.add(LeakyReLU(alpha = 0.1))
model.add(BatchNormalization())

model.add(Convolution2D(96, (3,3), padding='same', use_bias=False))
model.add(LeakyReLU(alpha = 0.1))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2, 2)))

model.add(Convolution2D(128, (3,3),padding='same', use_bias=False))
# model.add(BatchNormalization())
model.add(LeakyReLU(alpha = 0.1))
model.add(BatchNormalization())

model.add(Convolution2D(128, (3,3),padding='same', use_bias=False))
model.add(LeakyReLU(alpha = 0.1))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2, 2)))

model.add(Convolution2D(256, (3,3),padding='same',use_bias=False))
model.add(LeakyReLU(alpha = 0.1))
model.add(BatchNormalization())

model.add(Convolution2D(256, (3,3),padding='same',use_bias=False))
model.add(LeakyReLU(alpha = 0.1))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2, 2)))

model.add(Convolution2D(512, (3,3), padding='same', use_bias=False))
model.add(LeakyReLU(alpha = 0.1))
model.add(BatchNormalization())

model.add(Convolution2D(512, (3,3), padding='same', use_bias=False))
model.add(LeakyReLU(alpha = 0.1))
model.add(BatchNormalization())


model.add(Flatten())
model.add(Dense(512,activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(30))
#model.summary()













Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [ ]:
model = Sequential()

model.add(Convolution2D(32, (3,3), padding='same', use_bias=False, input_shape=(96,96,1)))
model.add(LeakyReLU(alpha = 0.1))
model.add(BatchNormalization())

model.add(Convolution2D(32, (3,3), padding='same', use_bias=False))
model.add(LeakyReLU(alpha = 0.1))
model.add(BatchNormalization())

model.add(Flatten())
model.add(Dense(128, activation="relu"))
model.add(Dropout(0.1))
model.add(Dense(64, activation="relu"))
model.add(Dense(30))
                        

In [ ]:
model.compile(optimizer='adam', 
              loss='mean_squared_error',
              metrics=['mae'])

In [ ]:
es = [EarlyStopping(monitor='val_loss', mode='min', verbose=3, patience=300), ModelCheckpoint(filepath='best_model.h5', monitor='val_loss', save_best_only=True)]
model.fit(X_train, y_train, epochs = 30, batch_size = 256, validation_split = 0.2, callbacks=es)



Train on 5639 samples, validate on 1410 samples
Epoch 1/30
5639/5639 [==============================] - 244s 43ms/step - loss: 736.3566 - mean_absolute_error: 19.8568 - val_loss: 184.6800 - val_mean_absolute_error: 10.7896
Epoch 2/30
5639/5639 [==============================] - 236s 42ms/step - loss: 151.5024 - mean_absolute_error: 8.6441 - val_loss: 101.4694 - val_mean_absolute_error: 8.1276
Epoch 3/30
5639/5639 [==============================] - 238s 42ms/step - loss: 120.2594 - mean_absolute_error: 7.6356 - val_loss: 196.7990 - val_mean_absolute_error: 11.6568
Epoch 4/30
5639/5639 [==============================] - 236s 42ms/step - loss: 81.5754 - mean_absolute_error: 6.5561 - val_loss: 451.7161 - val_mean_absolute_error: 18.1068
Epoch 5/30
5639/5639 [==============================] - 236s 42ms/step - loss: 74.5694 - mean_absolute_error: 6.3133 - val_loss: 119.1579 - val_mean_absolute_error: 8.8733
Epoch 6/30
5639/5639 [==============================] - 236s 42ms/step - loss: 60.1

# Test

In [ ]:
# Prédiction et submission

#preparing test data
timag = []
for i in range(0,1783):
    timg = test['Image'][i].split(' ')
    timg = ['0' if x == '' else x for x in timg]
    
    timag.append(timg)

timage_list = np.array(timag,dtype = 'float')
X_test = timage_list.reshape(-1,96,96,1) 

#On charge le meilleur modèle enregistré
model = load_model('best_model.h5')

pred = model.predict(X_test)

lookid_data = pd.read_csv('IdLookupTable.csv')
lookid_list = list(lookid_data['FeatureName'])
imageID = list(lookid_data['ImageId']-1)
pre_list = list(pred)

rowid = lookid_data['RowId']
rowid=list(rowid)

feature = []
for f in list(lookid_data['FeatureName']):
    feature.append(lookid_list.index(f))


preded = []
for x,y in zip(imageID,feature):
    preded.append(pre_list[x][y])



rowid = pd.Series(rowid,name = 'RowId')

loc = pd.Series(preded,name = 'Location')

submission = pd.concat([rowid,loc],axis = 1)

submission.to_csv('face_key_detection_submission.csv',index = False)